In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import itertools
import warnings
warnings.filterwarnings("ignore")
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
 # Veriyi yükleyin
df = pd.read_excel('/content/data/tum_magazalar.xlsx')

# Tarih sütununu index yapma
#df.set_index('Tarih', inplace=True)
df_monthly=df.groupby(['DepoRef']).resample('M', on='Tarih').agg({
    'Ciro': 'sum',
    'HaftaSonu': 'sum',
    'BayramEtkisi': 'mean',
    'CalismaGunSayisi': 'sum'
}).reset_index()
df_monthly['BayramEtkisi']=df_monthly['BayramEtkisi'].fillna(0)
df_monthly.set_index('Tarih', inplace=True)
def SARIMAX_train (DepoRef):

  df_monthly_depo=df_monthly[df_monthly['DepoRef']==DepoRef]
  df_monthly_depo=df_monthly_depo[['HaftaSonu', 'BayramEtkisi', 'CalismaGunSayisi','Ciro']]

  # StandardScaler'ı tanımlayın
  scaler = StandardScaler()
  # 'Ciro' sütununu standartlaştırın
  df_monthly_depo['Ciro'] = scaler.fit_transform(df_monthly_depo[['Ciro']])

  # En iyi sonucu saklamak için değişkenler
  best_score, best_params, best_seasonal_params, best_method, best_trend = float("inf"), None, None, None, None

  test = df_monthly_depo[(df_monthly_depo.index >= "2024-01") & ( df_monthly_depo.index <=  df_monthly_depo.index.max() )]
  exog_test = test[['HaftaSonu', 'BayramEtkisi', 'CalismaGunSayisi']]

  # Eğitim dönemi: 2022 ve 2023
  train =df_monthly_depo[df_monthly_depo.index < '2024-01']
  exog_train = train[['HaftaSonu', 'BayramEtkisi', 'CalismaGunSayisi']]

  next_date = pd.to_datetime(df_monthly.index.max(), format='%Y-%m') + relativedelta(months=1)
  next_date_str = next_date.strftime('%Y-%m')
  pred_ex = pd.DataFrame({
          "HaftaSonu": [6],
          "BayramEtkisi": [0.2666],
          "CalismaGunSayisi": [30]
      }, index=[pd.Timestamp(next_date_str)])
  df_monthly_depo = pd.concat([df_monthly_depo,pred_ex])


  # Tahmin edilecek: Mart 2025
  future = df_monthly_depo[df_monthly_depo.index == next_date_str]
  exog_future = future[['HaftaSonu', 'BayramEtkisi', 'CalismaGunSayisi']]

  methods = ['lbfgs', 'bfgs']
  trends = [ 'c', 'ct']

  # SARIMA için (p,d,q) parametreleri
  p = d = q = range(0, 2)
  pdq = list(itertools.product(p, d, q))
  # Mevsimsel parametreler (P,D,Q,s) s=12
  seasonal_pdq = [(x[0], x[1], x[2],12) for x in itertools.product(range(0, 2), range(0, 2), range(0, 2))]

  best_score, best_params, best_seasonal_params, best_method, best_trend = float("inf"), None, None, None, None

  param_grid = list(itertools.product(pdq, seasonal_pdq, methods, trends))

  for param_set in tqdm(param_grid):
    param, seasonal_param, method, trend = param_set
    try:
        # Model eğitimi
        model = SARIMAX(train['Ciro'],
                        exog=exog_train,
                        order=param,
                        seasonal_order=seasonal_param,
                        trend=trend,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        results = model.fit(method=method, disp=False)

        # Test dönemi boyunca adım adım tahmin
        history_end = train.copy()
        exog_history_end = exog_train.copy()
        errors = []

        for i in range(len(test)):
            # Her seferinde geçmişi bir ay uzat
            model = SARIMAX(history_end['Ciro'],
                            exog=exog_history_end,
                            order=param,
                            seasonal_order=seasonal_param,
                            trend=trend,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            results = model.fit(method=method, disp=False)

            # 1 adım tahmin
            forecast = results.get_forecast(steps=1, exog=exog_test.iloc[[i]])
            y_pred = forecast.predicted_mean.iloc[0]
            y_true = test['Ciro'].iloc[i]
            errors.append((y_pred - y_true) ** 2)

            # Gerçek değeri ve exog veriyi modele ekle
            new_row = test.iloc[[i]]
            history_end = pd.concat([history_end, new_row])
            exog_history_end = pd.concat([exog_history_end, exog_test.iloc[[i]]])

        # Ortalama MSE
        avg_mse = np.mean(errors)

        # En iyiyse güncelle
        if avg_mse < best_score:
            best_score = avg_mse
            best_params = param
            best_seasonal_params = seasonal_param
            best_method = method
            best_trend = trend

    except Exception as e:
        print(f"Hata: {e}")
        continue

  # En iyi modeli tekrar eğitip Mart 2025 için tahmin yap
  print("\nEn iyi model parametreleri:")
  print(f"order={best_params}, seasonal_order={best_seasonal_params}, method={best_method}, trend={best_trend}")
  print(f"Ortalama Test MSE: {best_score:.2f}")



  # Model eğitimi
  final_train = df_monthly_depo[df_monthly_depo.index < df_monthly_depo.index.max()]
  exog_final_train = final_train[['HaftaSonu', 'BayramEtkisi', 'CalismaGunSayisi']]


  final_model = SARIMAX(final_train['Ciro'],
                        exog=exog_final_train,
                        order=best_params,
                        seasonal_order=best_seasonal_params,
                        trend=best_trend,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
  final_results = final_model.fit(method=best_method, disp=False)


  """next_date = pd.to_datetime(df_monthly.index.max(), format='%Y-%m') + relativedelta(months=1)
  next_date_str = next_date.strftime('%Y-%m')
  pred_ex = pd.DataFrame({
          "Tarih": next_date_str ,
          "BayramEtkisi": [0.2666],
          "HaftaSonu": [6],
          "GunSayisi": [30]
      })
  pred_ex.set_index("Tarih", inplace=True)"""

  # Tahmin
  forecast = final_results.get_forecast(steps=1, exog=exog_future)
  next_month_forecast = forecast.predicted_mean.iloc[0]
  next_month_forecast_original = scaler.inverse_transform([[next_month_forecast]])[0][0]
  print(f"SARIMAX Model ----> According to {best_params},{best_seasonal_params}, {best_trend} and {best_method}, prediction for {DepoRef}: {next_month_forecast_original}")
  return best_score, best_params, best_seasonal_params, best_method, best_trend, next_month_forecast_original
  results = []
for i in df['DepoRef'].unique():
  print("{} için çalıştılıyor." .format(i))
  best_score, best_params, best_seasonal_params, best_method,best_trend,next_month_forecast_original=SARIMAX_train(i)
  results.append({
        'DepoRef': i,
        'Best Params': best_params,
        'Best Seasonal Params': best_seasonal_params,
        'Best Trend': best_trend,
        'Best Method': best_method,
        'Next Month Forecast': next_month_forecast_original
    })
